In [36]:
from selenium import webdriver
import json
from bs4 import BeautifulSoup


# Set up the browser (Chrome in headless mode)
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/114.0.0.0 Safari/537.36")
driver = webdriver.Chrome(options=options)

url = "https://leetcode.com/studyplan/top-100-liked/"

# Load the page
driver.get(url)

# Extract the fully rendered HTML
html = driver.page_source

rs = BeautifulSoup(html, "html.parser")

content = rs.find('script', id='__NEXT_DATA__').text


data = json.loads(content)

plan_details = {}

main = data['props']['pageProps']['dehydratedState']['queries'][0]['state']['data']['studyPlanV2Detail']

plan_details['slug'] = main['slug']
plan_details['name'] = main['name']
plan_details['problems'] = []

for section in main['planSubGroups']:
    s = {}
    s['name'] = section['name']
    s['size'] = section['questionNum']
    s['problems'] = []
    for problem in section['questions']:
        p = {}
        p['slug'] = problem['titleSlug']
        p['name'] = problem['title']
        p['difficulty'] = problem['difficulty']
        p['url'] = f"https://leetcode.com/problems/{problem['titleSlug']}"
        p['tags'] = [tag['name'] for tag in problem['topicTags']]
        s['problems'].append(p)
    plan_details['problems'].append(s)

with open( f'./study-plans/{plan_details['slug']}.json', 'w') as f:
    json.dump(plan_details, f, indent=4)


